In [1]:
#Let's import stuff
from theano.sandbox import cuda
%matplotlib inline

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
import utils; reload(utils)
from utils import *
from __future__ import division, print_function
from vgg16 import Vgg16

Using Theano backend.


In [3]:
#Let's load the dogs cats data
path = "data/dogscats/"
#saved model path
model_path = path + 'models/'


batch_size = 64

In [4]:
#now we create the model vgg,set the model as wellthen fine tune, but maybe add DA at the same time and batch normaliztion?
#Let's just create the model first and summary:
vgg = Vgg16()
model = vgg.model


In [5]:
model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [6]:
#Ok, since we want to retrain the last layer (layer 2), let's instead pop it and retain a new layer
model.pop()


In [7]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [8]:
#we need to make it so that we don't retrain anything yet.
for layer in model.layers: layer.trainable =False

In [9]:
#Double check not trainable now
for layer in model.layers: print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [10]:
#we need to make it so that we don't retrain anything yet.
model.add(Dense(2, activation='softmax'))


In [11]:
#let's look again
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  0           zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [21]:
#now let's gather our training data, labels, and validation batches and labels
training_data = get_data(path+'train')
validation_data = get_data(path+'valid')


Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [22]:
training_data.shape


(23000, 3, 224, 224)

In [23]:
validation_data.shape

(2000, 3, 224, 224)

In [12]:
#Not sure if we need this, but let's get the labels of the 
validation_batches = get_batches(path+'valid', shuffle = False, batch_size = batch_size)
batches = get_batches(path+'train', shuffle = True, batch_size = batch_size)


Found 2000 images belonging to 2 classes.
Found 23000 images belonging to 2 classes.


In [24]:
print(vgg.classes)

[u'tench', u'goldfish', u'great_white_shark', u'tiger_shark', u'hammerhead', u'electric_ray', u'stingray', u'cock', u'hen', u'ostrich', u'brambling', u'goldfinch', u'house_finch', u'junco', u'indigo_bunting', u'robin', u'bulbul', u'jay', u'magpie', u'chickadee', u'water_ouzel', u'kite', u'bald_eagle', u'vulture', u'great_grey_owl', u'European_fire_salamander', u'common_newt', u'eft', u'spotted_salamander', u'axolotl', u'bullfrog', u'tree_frog', u'tailed_frog', u'loggerhead', u'leatherback_turtle', u'mud_turtle', u'terrapin', u'box_turtle', u'banded_gecko', u'common_iguana', u'American_chameleon', u'whiptail', u'agama', u'frilled_lizard', u'alligator_lizard', u'Gila_monster', u'green_lizard', u'African_chameleon', u'Komodo_dragon', u'African_crocodile', u'American_alligator', u'triceratops', u'thunder_snake', u'ringneck_snake', u'hognose_snake', u'green_snake', u'king_snake', u'garter_snake', u'water_snake', u'vine_snake', u'night_snake', u'boa_constrictor', u'rock_python', u'Indian_cob

In [28]:
#Get classes from the batches, set the class variable to the classes (this may not be needed)
classes = list(iter(batches.class_indices))
print(classes)
print(type(classes))
#This may be needed
##for c in batches.class_indices:
#   classes[batches.class_indices[c]] = c
vgg.classes = classes
print(vgg.classes)

['cats', 'dogs']
<type 'list'>
['cats', 'dogs']


In [29]:
#Convert to labels, after we one hot encode
validation_classes = validation_batches.classes
training_classes = batches.classes
validation_labels = onehot(validation_classes)
training_labels = onehot(training_classes)

In [30]:
training_labels.shape

(23000, 2)

In [31]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  0           zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [32]:
#now we get ready to fit
#gen=image.ImageDataGenerator(rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1, horizontal_flip = True)
gen = image.ImageDataGenerator()
batches = gen.flow(training_data, training_labels, batch_size=batch_size, shuffle=True)
validation_batches = gen.flow(validation_data, validation_labels, batch_size=batch_size, shuffle=False)

In [33]:
#Set our learning rates and optimizers
optim = RMSprop(lr = 0.0005)
model.compile(optimizer = optim, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [34]:
epoch_number = 3


In [35]:
def fit_model(model, batches, val_batches, nb_epoch=1):
    model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=nb_epoch, 
                        validation_data=val_batches, nb_val_samples=val_batches.N)

In [ ]:
fit_model(model, batches, validation_batches, nb_epoch = epoch_number)
#Here we go...

Epoch 1/3
23000/23000 [==============================] - 596s - loss: 0.1167 - acc: 0.9656 - val_loss: 0.0444 - val_acc: 0.9850

In [38]:
#wir sollen es bisschen mehr probieren...
fit_model(model, batches, validation_batches, nb_epoch = 1)


Epoch 1/1
23000/23000 [==============================] - 596s - loss: 0.0665 - acc: 0.9811 - val_loss: 0.0438 - val_acc: 0.9840

In [42]:
#One more time 
model.save_weights(model_path + 'stephen_a1_2nodataaug_try2.h5')

In [15]:
model.load_weights(model_path + 'stephen_a1_1.h5')

In [17]:
#validation_features = model.predict(validation_data, batch_size=batch_size)

In [43]:
######let's try and get some data out
predictions = model.predict_classes(validation_data, batch_size = batch_size)
wahrscheinlichkeit = model.predict_proba(validation_data, batch_size = batch_size)

2000/2000 [==============================] - 47s     

In [44]:
#now let's plot a confusion matrix...will this work?
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(validation_classes, predictions)
#Note, validation classes can sometimes be called expected labels

In [45]:
plot_confusion_matrix(cm, validation_batches.class_indices)

AttributeError: 'NumpyArrayIterator' object has no attribute 'class_indices'

In [ ]:
Predict 

In [46]:
########Now let's parse the data for submission to see how we did
##WE NEED TO GENERATE PREDICTIONS
batches = 0
test_path = path + 'test/'
batches = get_batches(path, shuffle=False, batch_size=batch_size, class_mode=None)
predictions = model.predict_generator(batches, batches.nb_sample)

#get filenames 
filenames = batches.filenames

Found 12500 images belonging to 6 classes.


In [47]:
results_path = model_path
#we save array predictions and filenames here:
save_array(results_path + 'stephen_test_predictions_a1_test2.dat', predictions)
save_array(results_path + 'stephen_test_filenames_a1_test2.dat', filenames)


In [48]:
#my_predictions = wahrscheinlichkeit[:,0]
#my_labels = np.round(1-my_predictions)
wahrscheinlichkeit = 0
is_a_hund = predictions[:,1]
is_a_hund.shape



(12500,)

In [49]:
#CLIPPING
is_a_hund = is_a_hund.clip(min=0.2, max=0.98)

In [50]:
filenames[:5]


['test1/9292.jpg',
 'test1/12026.jpg',
 'test1/9688.jpg',
 'test1/4392.jpg',
 'test1/779.jpg']

In [53]:
len(filenames)

12500

In [54]:
#ts = 'test1/43922342.jpg'
#newa = ts[ts.find('/')+1:ts.find('.jpg')]
#print(newa)
ids = np.array([int(ts[ts.find('/')+1:ts.find('.jpg')]) for ts in filenames])
#ids = np.array([int(ts[ts.find('.')+1:ts.find('.jpg')]) for ts in filenames])
#the above was from an incorrect implementation, but the parsing is good knowledge



In [55]:
ids.shape

(12500,)

In [56]:
stacked_together = np.stack([ids,is_a_hund], axis = 1)
stacked_together[:5]

array([[  9.2920e+03,   2.0000e-01],
       [  1.2026e+04,   2.0000e-01],
       [  9.6880e+03,   2.0000e-01],
       [  4.3920e+03,   2.0000e-01],
       [  7.7900e+02,   9.8000e-01]])

In [57]:
from IPython.display import FileLink
import os, sys
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/data/'
submission_file_name = 'stephen_submission1.csv'
np.savetxt(submission_file_name, stacked_together, fmt='%d,%.5f', header='id,label', comments='')
print(current_dir)

/home/ubuntu/courses/deeplearning1/nbs


In [58]:
FileLink(submission_file_name)

/home/ubuntu/courses/deeplearning1/nbs/stephen_submission1.csv

In [60]:
FileLink('MyDogsVsCats.ipynb')

/home/ubuntu/courses/deeplearning1/nbs/MyDogsVsCats.ipynb